<a href="https://colab.research.google.com/github/SLVmain/PyTorch_start/blob/Les_09/hw_09_AttentionTransformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашка к уроку 9

Переписать загрузку данных с python функций на Dataset и Dataloader и применить сеть с attention

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np


In [4]:
batch_size = 64
epochs = 30
latent_dim = 256
num_samples = 10000
data_path = '/content/drive/MyDrive/GB_Pytorch_01_2022/09_transformers/Lecture09-10/data/fra-eng/fra.txt'

In [5]:
# Собираем из текстов токены

input_texts = []
target_texts = []

input_vocab = set()
output_vocab = set()

with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
    #print(lines)
for line in lines[: min(num_samples, len(lines) - 1)]: # ограничеваем по кол-ву примеров
    input_text, target_text, _ = line.split('\t') # получаем фразы на англ и франц
    
    target_text = '\t' + target_text + '\n'  #фразы на франц с табуляцией и с новой строки
    
    input_texts.append(input_text)  #фразы на английском
    
    for word in input_text.split():
        input_vocab.add(word.strip()) #отдельно слова на англ

    target_texts.append(target_text)  #фразы на франц
    for word in target_text.split():
        output_vocab.add(word.strip()) #слова на франц
    
input_vocab2index = {word: i+2 for i, word in enumerate(input_vocab)} # словарь слово - номер, начиная с 2
output_vocab2index = {word: i+2 for i, word in enumerate(output_vocab)} # словарь слово - номер, начиная с 2

In [ ]:
#print(output_vocab2index)

In [ ]:
#print(target_texts)

In [11]:
def indexesFromSentence(sentence, vocab):
    return [vocab.get(word.strip(), 0) for word in sentence.split(' ')]

def tensorFromSentence(sentence, vocab):
    indexes = indexesFromSentence(sentence, vocab)
    indexes.append(1)
    return torch.tensor(indexes, dtype=torch.long).view(-1, 1)


def tensorsFromSent(input_sentences, output_sentences):
    input_tensor = tensorFromSentence(input_sentences, input_vocab2index)
    target_tensor = tensorFromSentence(output_sentences, output_vocab2index)
    return (input_tensor, target_tensor)

In [69]:
len(input_texts)

10000

In [ ]:
training_pairs = np.random.randint(0, len(input_texts), size=10000)
#print(training_pairs)

In [15]:
for i in range(1):

    input_tensor, target_tensor = tensorsFromSent(input_texts[training_pairs[i]], target_texts[training_pairs[i]])

input_tensor, target_tensor

(tensor([[2144],
         [ 271],
         [1622],
         [   1]]), tensor([[4906],
         [4622],
         [ 193],
         [5218],
         [4948],
         [   1]]))

In [27]:
input_tensor.shape, target_tensor.shape

(torch.Size([3, 1]), torch.Size([5, 1]))

In [17]:
input_tensor, target_tensor = tensorsFromSent(input_texts[4403], target_texts[4403])
input_tensor, target_tensor

(tensor([[1205],
         [2769],
         [   1]]), tensor([[4334],
         [2777],
         [ 632],
         [5243],
         [   1]]))

In [6]:
class AttDataset(torch.utils.data.Dataset):
    
    def __init__(self, input_vocab2index, output_vocab2index, input_texts, target_texts):
        self.input_vocab2index = input_vocab2index
        self.output_vocab2index = output_vocab2index
        self.input_texts = input_texts
        self.target_texts = target_texts

    def __len__(self):
        return len(self.input_texts)

    def indexesFromSentence(self, sentence, vocab):
        return [vocab.get(word.strip(), 0) for word in sentence.split(' ')]

    def tensorFromSentence(self, sentence, vocab):
        indexes = self.indexesFromSentence(sentence, vocab)
        indexes.append(1)
        return torch.tensor(indexes, dtype=torch.long).view(-1, 1)

    def tensorsFromSent(self, input_sentences, output_sentences):
        input_tensor = self.tensorFromSentence(input_sentences, self.input_vocab2index)
        target_tensor = self.tensorFromSentence(output_sentences, self.output_vocab2index)
        return (input_tensor, target_tensor)
    
    def __getitem__(self, idx):
        
        input_tensor, target_tensor = self.tensorsFromSent(input_texts[idx], target_texts[idx])
        
        return (input_tensor, target_tensor)



In [7]:
train_dataset = AttDataset(input_vocab2index, output_vocab2index, input_texts, target_texts)         


data_loader = torch.utils.data.DataLoader(train_dataset,
                                          batch_size=1, #???
                                          shuffle=True,
                                          num_workers=2)

In [ ]:
'''dataset = AttDataset(input_vocab2index, output_vocab2index, input_texts, target_texts)  
for idx, (in_, out_) in enumerate(dataset):
    print(idx)'''

In [24]:
it = iter(data_loader)
first = next(it)
second = next(it)

print(first)

[tensor([[[1740],
         [1114],
         [   1]]]), tensor([[[2381],
         [ 210],
         [3150],
         [ 733],
         [   1]]])]


In [22]:
print(first[0].shape) #размер input tensor

torch.Size([1, 5, 1])


In [25]:
print(second[0].shape)

torch.Size([1, 4, 1])


In [36]:
a = torch.squeeze(second[0], 0)
a.shape

torch.Size([4, 1])

In [8]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input_, hidden):
        embedded = self.embedding(input_).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size)

In [9]:
class DecoderRNN(nn.Module):
    
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, input_size)
        
        
    def forward(self, input_, hidden, encoder_outputs):
        embedded = self.embedding(input_).view(1, 1, -1)
        output, hidden = self.lstm(embedded, hidden.view(1, 1, 1, -1))
        output = F.log_softmax(self.out(output[0]), dim=1)
        return output

In [10]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=10):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        #output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size)

In [11]:
def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=10):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[0]])

    decoder_hidden = encoder_hidden

    for di in range(target_length):
        decoder_output, decoder_hidden, decoder_attention = decoder(
            decoder_input, decoder_hidden, encoder_outputs)
        topv, topi = decoder_output.topk(1)
        decoder_input = topi.squeeze().detach()  # detach from history as input

        loss += criterion(decoder_output, target_tensor[di])
        if decoder_input.item() == 1:
            break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [14]:
encoder = EncoderRNN(len(input_vocab2index)+2, 30)
attn_decoder1 = AttnDecoderRNN(30, len(output_vocab2index)+2, dropout_p=0.1)

#attn_decoder1 = DecoderRNN(len(output_vocab2index)+2, 30)

encoder_optimizer = torch.optim.SGD(encoder.parameters(), lr=0.01)
decoder_optimizer = torch.optim.SGD(attn_decoder1.parameters(), lr=0.01)
#training_pairs = np.random.randint(0, len(input_texts), size=10000)
criterion = nn.NLLLoss()

print_loss_total = 0
#for i in range(1000):
for i, data in enumerate(data_loader):
    #input_tensor, target_tensor = tensorsFromSent(input_texts[training_pairs[i]], target_texts[training_pairs[i]])
    input_tensor, target_tensor = data
    input_tensor = torch.squeeze(input_tensor, 0)
    target_tensor = torch.squeeze(target_tensor, 0)
    loss = train(input_tensor, target_tensor, encoder,
               attn_decoder1, encoder_optimizer, decoder_optimizer, criterion)
    print_loss_total += loss
    
    print_loss_avg = print_loss_total / 1
    print_loss_total = 0
    print('(%d %d%%) %.4f' % (i, i / 10 * 100, print_loss_avg))


Выходные данные были обрезаны до нескольких последних строк (5000).
(5000 50000%) 5.6686
(5001 50010%) 4.3777
(5002 50020%) 6.8815
(5003 50030%) 6.8991
(5004 50040%) 4.6106
(5005 50050%) 4.6995
(5006 50060%) 2.5953
(5007 50070%) 6.0423
(5008 50080%) 4.4926
(5009 50090%) 4.7630
(5010 50100%) 3.2574
(5011 50110%) 4.2210
(5012 50120%) 4.2050
(5013 50130%) 3.1981
(5014 50140%) 1.4908
(5015 50150%) 6.7408
(5016 50160%) 4.5897
(5017 50170%) 5.9554
(5018 50180%) 4.2440
(5019 50190%) 5.2213
(5020 50200%) 3.0070
(5021 50210%) 3.9528
(5022 50220%) 4.2539
(5023 50230%) 3.4849
(5024 50240%) 5.6370
(5025 50250%) 4.0562
(5026 50260%) 4.8874
(5027 50270%) 3.4448
(5028 50280%) 5.3379
(5029 50290%) 4.5304
(5030 50300%) 5.8417
(5031 50310%) 4.5314
(5032 50320%) 5.0799
(5033 50330%) 4.5938
(5034 50340%) 2.1967
(5035 50350%) 4.4118
(5036 50360%) 5.1613
(5037 50370%) 4.5156
(5038 50380%) 4.6981
(5039 50390%) 2.8967
(5040 50400%) 1.2644
(5041 50410%) 2.8732
(5042 50420%) 5.5881
(5043 50430%) 4.0488
(5044 50